In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# seaborne is a statistical plotting package 
# https://seaborn.pydata.org/
import seaborn as sns 
#sns.set(sytle="darkgrid")
# use grid layout functions. 
import matplotlib.gridspec as gridspec
# a bunch of ueful functions for plotting data https://matplotlib.org/stable/api/mlab_api.html
import matplotlib.mlab as mlab
# Setup everything here. 

# 
# 27-Dec-21  CBL state space modeling of classical processes.
#

#
# Add in a figure that we can plot to real time as we update the state
#

In [6]:
# Make State Space Class - Sorry introducting classes now. 
class StateSpace ():
    def __init__(self):
        # Time step
        dt = 0.05
        #
        # Matrix to describe the process
        # 
        # Initial Position, make sure that it is a numpy style array and not a python list. 
        self.X0 = np.array([0, 0, 0], dtype=np.float)
        # Velocity (could be speed and heading theta = 90-H)
        self.V0 = np.array([1.0, 0, 0], dtype=np.float)
        # Acceleration
        self.A0 = np.array([1.0, 0.0, 0.0], dtype=np.float)
        #
        # 9x9 state matrix advance the state
        # Use matrix because it does matrix math. 
        # arrays do element by element multiplication with the operator *
        # Could use array and then use the np.dot(A,B) operator
        #
        self.P = np.matrix([
                    [1.0, 0.0, 0.0,  dt, 0.0, 0.0, dt*dt,   0.0,   0.0],
                    [0.05, 1.0, 0.0, 0.0,  dt, 0.0,   0.0, dt*dt,   0.0],
                    [0.0, 0.0, 1.0, 0.0, 0.0,  dt,   0.0,   0.0, dt*dt],
                    [0.0, 0.0, 0.0, 1.0, 0.0, 0.0,    dt,   0.0,   0.0],
                    [0.0, 0.0, 0.0, 0.0, 1.0, 0.0,   0.0,    dt,   0.0],
                    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0,   0.0,   0.0,    dt],
                    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0,   1.0,   0.0,   0.0],
                    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0,   0.0,   1.0,   0.0],
                    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0,   0.0,   0.0,   1.0]],dtype=float)
    def Zero(self):
        # reinitialize the system
        # Initialize full X vector
        # while I use this format here, I suppose I could have it as a row array and do a transpose. 
        self.X = np.matrix([[self.X0[0]], [self.X0[1]], [self.X0[2]], 
                           [self.V0[0]], [self.V0[1]], [self.V0[2]], 
                           [self.A0[0]], [self.A0[1]], [self.A0[2]]], dtype=float)
    def Advance(self):
        # repeat this call for continued advancement of the state. 
        self.X = self.P * self.X
        print(self.X)
        
    def GetX(self):
        return self.X[0]
    def GetY(self):
        return self.X[1]

In [3]:
class PositionPlot():
    def __init__(self):
        self.__NPoints    = 20   # how often we expand memory if necessary.  
        self.__xdata      = np.zeros(self.__NPoints) 
        self.__ydata      = np.zeros(self.__NPoints) 
        self.__PointCount = 0    # number of points entered to date. 
        self.__binwidth   = 0.1  # histogram bin size. 
        self.__MinX       = -1.0 
        self.__MaxX       =  1.0 
        self.__MinY       = -1.0 
        self.__MaxY       =  1.0 
        self.__MyPlot     = [] 
        self.__Error      = False 
        self.__CodeVersion= 0.5 
        # 
        # define the layout of the scatterplot and histograms. 
        # 
        # definitions for the axes 
        left, width    = 0.1, 0.65 
        bottom, height = 0.1, 0.65 
        spacing        = 0.005 
        rect_scatter   = [left, bottom, width, height] 
        rect_histx     = [left, bottom + height + spacing, width, 0.2] 
        rect_histy     = [left + width + spacing, bottom, 0.2, height] 
         
        # start with a square Figure: 8x8 inches 
        #  https://matplotlib.org/api/_as_gen/matplotlib.figure.Figure.html 
        fig = plt.figure(figsize=(8, 8)) 
        # Adding these bits helps get the plot always showing.  
        fig.canvas.draw() 
        plt.show(block=False) 
 
        # 
        # create the central figure for the scatter plot 
        # 
        # 
        self.__ax = fig.add_axes(rect_scatter) 
        self.__ax.grid(True) 
        #self.__ax.set_title('GPS X-Y drift') 
        self.__ax.set_xlabel('Centered False Easting') 
        self.__ax.set_ylabel('Centered False Northing') 
        self.__ax.set_xlim(self.__MinX, self.__MaxX) 
        self.__ax.set_ylim(self.__MinY, self.__MaxY)
                                                                            
        # 
        # create a histogram for the x projection 
        # 
        self.__ax_histx = fig.add_axes(rect_histx, sharex=self.__ax) 
        self.__ax_histx.grid(True) 
 
        # 
        # create a histogram for the y projection 
        # 
        self.__ax_histy = fig.add_axes(rect_histy, sharey=self.__ax) 
        self.__ax_histy.grid(True) 
        
    def BlockFill(self, x, y):
        """@brief Method to update the current points in the plot. 
        This entry point has all the points. 
        NOTE: The self.x and self.y are not updated.  
        @param x - vector of x points to update 
        @param y - vector of y points to update 
        """ 
        self.__Error = False 
 
        # no labels 
        self.__ax_histx.tick_params(axis="x", labelbottom=False)            
        self.__ax_histy.tick_params(axis="y", labelleft=False) 
 
        # 
        # the scatter plot, using the plot command 
        # since I won't use multiple symbols/markers and this 
        # is substantially faster. 
        # 
        self.__MyPlot = self.__ax.plot(x, y, 'g.') 
        # now determine nice limits by hand: 
        xymax = max(np.max(np.abs(x)), np.max(np.abs(y))) 
        lim = (int(xymax/self.__binwidth) + 1) * self.__binwidth 
 
        bins = np.arange(-lim, lim + self.__binwidth, self.__binwidth) 
        self.__ax_histx.hist(x, bins=bins,color='b') 
        self.__ax_histy.hist(y, bins=bins, orientation='horizontal', color='b'\
)
    def Mean(self): 
        """ 
        @brief return the mean of the x and y axis based on the 
        points currently in the scatter plot. 
        """ 
        self.__Error = False 
        if (self.__MyPlot): 
            # A plot does exist to extract data from.  
            x,y = self.__MyPlot[0].get_data() 
            return np.mean(x),np.mean(y) 
        else: 
            self.__Error = True 
            return -9999999, -9999999 

    def Std(self): 
        """ 
        @brief return the std of the x and y axis based on the 
        points currently in the scatter plot. 
        """ 
        self.__Error = False 
        if (self.__MyPlot): 
            # A plot does exist to extract data from.  
            x,y = self.__MyPlot[0].get_data() 
            return np.std(x),np.std(y) 
        else: 
            self.__Error = True 
            return -9999999, -9999999 
         
    def update(self, x, y): 
        """@brief Method to update the current points in the plot. 
        the previous instantiation updated a single block at a time 
        rather than using a point pair. 
        @param x - a single point to update 
        @param y - a single point to update 
        """ 
        self.__Error = False 
         
        # increment the number of points in the plot.                       
        self.__PointCount += 1 
        if (self.__PointCount >= self.__NPoints): 
            # increase the size of the arrays.  
            self.__xdata = np.append(self.__xdata, np.zeros(self.__NPoints)) 
            self.__ydata = np.append(self.__ydata, np.zeros(self.__NPoints)) 
        # 
        # Now it is big enough to hold the requested point set. 
        # 
        self.__xdata[self.__PointCount] = x 
        self.__ydata[self.__PointCount] = y 
        # 
        # now do the block update. 
        # 
        self.BlockFill(self.__xdata[0:self.__PointCount], 
                       self.__ydata[0:self.__PointCount]) 
 
    def draw(self): 
        """ 
        @brief update the plot on the screen. 
        """ 
        self.__ax.figure.canvas.draw() 
        self.AddStatistics() 
        self.__ax.figure.canvas.flush_events() 
         
    def refresh(self): 
        """ 
        @brief do a full refresh of the drawing. 
        """ 
        # This does not appear to work.  
        self.__ax.figure.canvas.draw() 
        plt.show(block=False) 

    def AddStatistics(self):
        """
        @brief Get the statistics about the plot and put them in a
        text box that the user can read the basic numbers.
        """
        muX,muY = self.Mean()
        sigmaX,sigmaY = self.Std()
        str1 = ' '.join((r'$\mu_X=%.2f$' % (muX, ),
                        r'$\sigma_X=%.2f$' % (sigmaX, )))
        str2 = ' '.join((r'$\mu_Y=%.2f$' % (muY, ),
                        r'$\sigma_Y=%.2f$' % (sigmaY, )))

        textstr = '\n'.join((str1, str2))
##        textstr = '\n'.join((
##            r'$\mu_X=%.2f$' % (muX, ), r'$\sigma_X=%.2f$' % (sigmaX, ),
##            r'$\mu_Y=%.2f$' % (muY, ),
##            r'$\sigma_Y=%.2f$' % (sigmaY, )))

        # these are matplotlib.patch.Patch properties
    
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

        # place a text box in upper left in axes coords
        self.__ax.text(0.05, 0.95, textstr,
                       transform=self.__ax.transAxes, fontsize=14,
                       verticalalignment='top', bbox=props)

    def IsError(self):
        """
        @brief returns true if an error occured during the last call.
        """
        return self.__Error

    def show(self):
        plt.show()

    def Version(self):
        return self.__CodeVersion

In [8]:
# declare the class
ss = StateSpace()
# setup plotting
PP = PositionPlot()
# initiaize the X state space vectror
ss.Zero()
# advance the vector, as you see in the definition, it prints the output.
ss.Advance()
# plot the position
PP.update(ss.GetX(),ss.GetY())
PP.draw()
# 27-Dec-21 The PP drawing does not work in this setup. 

<Figure size 576x576 with 0 Axes>

[[0.0525]
 [0.    ]
 [0.    ]
 [1.05  ]
 [0.    ]
 [0.    ]
 [1.    ]
 [0.    ]
 [0.    ]]
